In [19]:
# 同庄老师课堂所说
import torch
import torch.nn as nn
from torch.utils import data
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import confusion_matrix
import torchvision
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import MultiStepLR, CosineAnnealingLR
from torch.nn import Module, Linear, Dropout, LayerNorm, Identity
import numpy as np
import torch
import torch.nn as nn
import torchvision
import logging
import pandas as pd
import numpy as np
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torch import optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
import time

In [20]:
def adjust_learning_rate(optimizer, epoch ):
    import math
    lr = 1e-4
    Epochs = 200
    warmup = 5

    if epoch < warmup:
        lr = lr / (warmup - epoch)
    else:
        lr *= 0.5 * (1. + math.cos(math.pi * (epoch - warmup) / (Epochs - warmup)))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
print('okk')

okk


In [21]:
def cutmix(batch,alpha=1):
    inputs,labels = batch
    indices = torch.randperm(inputs.size(0))
    shuffled_inputs = inputs[indices]
    shuffled_labels = labels[indices]

    weight,height= inputs.shape[2:]
    lamb = np.random.beta(alpha, alpha)

    rx=np.random.uniform(0, weight)
    ry=np.random.uniform(0, height)

    rw=weight*np.sqrt(1-lamb)
    rh=height*np.sqrt(1-lamb)

    x0 = int(np.round(max(rx - rw / 2, 0)))
    x1 = int(np.round(min(rx + rw / 2, weight)))
    y0 = int(np.round(max(ry - rh / 2, 0)))
    y1 = int(np.round(min(ry + rh / 2, height)))

    inputs[:, :, y0:y1, x0:x1] = shuffled_inputs[:, :, y0:y1, x0:x1]
    labels = (labels, shuffled_labels, lamb)

    return inputs,labels

class CutMixCollator:
    def __init__(self, alpha):
        self.alpha = alpha

    def __call__(self, batch):
        batch = torch.utils.data.dataloader.default_collate(batch)
        batch = cutmix(batch, self.alpha)
        return batch

class CutMixCriterion:
    def __init__(self, reduction):
        self.criterion = nn.CrossEntropyLoss(reduction=reduction)

    def __call__(self, preds, targets):
        targets1, targets2, lam = targets
        return lam * self.criterion(
            preds, targets1) + (1 - lam) * self.criterion(preds, targets2)

In [22]:
def get_loader(batch_size=128,num_workers=4,cutmix=True,alpha=1):
    if cutmix==True:
        collator = CutMixCollator(alpha)
    else:
        collator = torch.utils.data.dataloader.default_collate

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    train_dataset = datasets.CIFAR10(root='cifar', train=True, transform=transforms.Compose([transforms.RandomHorizontalFlip(),
                    transforms.RandomCrop(32, 4),
                    transforms.ToTensor(),
                    normalize,]), download=True)
      
    dev_dataset=datasets.CIFAR10(root='cifar', train=False, transform=transforms.Compose([                                                                    
                transforms.ToTensor(),
                normalize,]))

    train_dataloader = torch.utils.data.DataLoader(train_dataset,      
                       batch_size=batch_size, shuffle=True,
                       num_workers=num_workers, pin_memory=True,
                       drop_last=True,collate_fn=collator)
    
    dev_dataloader = torch.utils.data.DataLoader( dev_dataset, 
                     batch_size=batch_size, shuffle=False,
                     num_workers=num_workers, pin_memory=True,
                     drop_last=False)
    return train_dataloader,dev_dataloader

In [23]:
train_dataloader,dev_dataloader=get_loader(batch_size=128,num_workers=4,cutmix=True,alpha=1)

Files already downloaded and verified


In [24]:
# *------------------- CCT: Transformer -------------------------*
# Meta CNN, CCT transformer
class Attention(Module):
    """
    Obtained from: github.com:rwightman/pytorch-image-models
    """

    def __init__(self, dim, num_heads=8, attention_dropout=0.1, projection_dropout=0.1):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // self.num_heads
        self.scale = head_dim ** -0.5

        self.qkv = Linear(dim, dim * 3, bias=False)
        self.attn_drop = Dropout(attention_dropout)
        self.proj = Linear(dim, dim)
        self.proj_drop = Dropout(projection_dropout)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        # 3, Batch , num_heads , N(seq_len) , channels // num_heads (scale)
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class TransformerEncoderLayer(Module):
    """
    Inspired by torch.nn.TransformerEncoderLayer and
    rwightman's timm package.
    """

    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.1,
                 attention_dropout=0.1, drop_path_rate=0.1):
        super(TransformerEncoderLayer, self).__init__()
        self.pre_norm = LayerNorm(d_model)
        self.self_attn = Attention(dim=d_model, num_heads=nhead,
                                   attention_dropout=attention_dropout, projection_dropout=dropout)

        self.linear1 = Linear(d_model, dim_feedforward)
        self.dropout1 = Dropout(dropout)
        self.norm1 = LayerNorm(d_model)
        self.linear2 = Linear(dim_feedforward, d_model)
        self.dropout2 = Dropout(dropout)

        self.drop_path = DropPath(drop_path_rate) if drop_path_rate > 0 else Identity()

        self.activation = F.gelu

    def forward(self, src: torch.Tensor, *args, **kwargs) -> torch.Tensor:
        src = src + self.drop_path(self.self_attn(self.pre_norm(src)))
        src = self.norm1(src)
        src2 = self.linear2(self.dropout1(self.activation(self.linear1(src))))
        src = src + self.drop_path(self.dropout2(src2))
        return src


def drop_path(x, drop_prob: float = 0., training: bool = False):
    """
    Obtained from: github.com:rwightman/pytorch-image-models
    Drop paths (Stochastic Depth) per sample (when applied in main path of residual blocks).
    This is the same as the DropConnect impl I created for EfficientNet, etc networks, however,
    the original name is misleading as 'Drop Connect' is a different form of dropout in a separate paper...
    See discussion: https://github.com/tensorflow/tpu/issues/494#issuecomment-532968956 ... I've opted for
    changing the layer and argument names to 'drop path' rather than mix DropConnect as a layer name and use
    'survival rate' as the argument.
    """
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # work with diff dim tensors, not just 2D ConvNets
    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
    random_tensor.floor_()  # binarize
    output = x.div(keep_prob) * random_tensor
    return output


class DropPath(Module):
    """
    Obtained from: github.com:rwightman/pytorch-image-models
    Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks).
    """

    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)


class Tokenizer(nn.Module):
    def __init__(self,
                 kernel_size, stride, padding,
                 pooling_kernel_size=3, pooling_stride=2, pooling_padding=1,
                 n_conv_layers=1,
                 n_input_channels=3,
                 n_output_channels=64,
                 in_planes=64,
                 activation=None,
                 max_pool=True):
        super(Tokenizer, self).__init__()
        # 不用麻烦了，就搞一层
        n_filter_list = [n_input_channels] + \
                        [in_planes for _ in range(n_conv_layers - 1)] + \
                        [n_output_channels]
        # input -- 中间过渡 in_planes * in_planes -- output channel

        self.conv_layers = nn.Sequential(
            *[nn.Sequential(
                nn.Conv2d(n_filter_list[i], n_filter_list[i + 1],
                          kernel_size=(kernel_size, kernel_size),
                          stride=(stride, stride),
                          padding=(padding, padding), bias=False),
                nn.Identity() if activation is None else activation(),
                nn.MaxPool2d(kernel_size=pooling_kernel_size,
                             stride=pooling_stride,
                             padding=pooling_padding) if max_pool else nn.Identity()
            )
                for i in range(n_conv_layers)
            ])

        self.flattener = nn.Flatten(2, 3)
        self.apply(self.init_weight)

    def sequence_length(self, n_channels=3, height=32, width=32):
        return self.forward(torch.zeros((1, n_channels, height, width))).shape[1]  # 你这，有点不优异，自己又跑了一遍可还行

    def forward(self, x):
        # return self.flattener(self.conv_layers(x)).transpose(-2, -1)
        out = self.conv_layers(x)  # 连着对 H、W downsample两次， 砍成1/4
        return self.flattener(out).transpose(-2, -1)  # [BS , downsample(H*W) , output C]

    @staticmethod
    def init_weight(m):
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight)


class TransformerClassifier(nn.Module):
    def __init__(self,
                 seq_pool=True,
                 embedding_dim=768,
                 num_layers=12,
                 num_heads=12,
                 mlp_ratio=4.0,
                 num_classes=1000,
                 dropout_rate=0.1,
                 attention_dropout=0.1,
                 stochastic_depth_rate=0.1,
                 positional_embedding='sine',
                 sequence_length=None,
                 *args, **kwargs):
        super().__init__()
        positional_embedding = positional_embedding if \
            positional_embedding in ['sine', 'learnable', 'none'] else 'sine'
        dim_feedforward = int(embedding_dim * mlp_ratio)
        self.embedding_dim = embedding_dim
        self.sequence_length = sequence_length
        self.seq_pool = seq_pool

        assert sequence_length is not None or positional_embedding == 'none', \
            f"Positional embedding is set to {positional_embedding} and" \
            f" the sequence length was not specified."

        if not seq_pool:
            sequence_length += 1
            self.class_emb = nn.Parameter(torch.zeros(1, 1, self.embedding_dim),
                                          requires_grad=True)
        else:
            self.attention_pool = nn.Linear(self.embedding_dim, 1)

        if positional_embedding != 'none':
            if positional_embedding == 'learnable':
                self.positional_emb = nn.Parameter(torch.zeros(1, sequence_length, embedding_dim),
                                                   requires_grad=True)
                nn.init.trunc_normal_(self.positional_emb, std=0.2)
            else:
                self.positional_emb = nn.Parameter(self.sinusoidal_embedding(sequence_length, embedding_dim),
                                                   requires_grad=False)
        else:
            self.positional_emb = None

        self.dropout = nn.Dropout(p=dropout_rate)
        dpr = [x.item() for x in torch.linspace(0, stochastic_depth_rate, num_layers)]
        self.blocks = nn.ModuleList([
            TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads,
                                    dim_feedforward=dim_feedforward, dropout=dropout_rate,
                                    attention_dropout=attention_dropout, drop_path_rate=dpr[i])
            for i in range(num_layers)])  # 几层 transformer
        self.norm = nn.LayerNorm(embedding_dim)

        self.fc = nn.Linear(embedding_dim, num_classes)
        self.apply(self.init_weight)

    def forward(self, x):
        if self.positional_emb is None and x.size(1) < self.sequence_length:
            x = F.pad(x, (0, 0, 0, self.n_channels - x.size(1)), mode='constant', value=0)

        if not self.seq_pool:
            cls_token = self.class_emb.expand(x.shape[0], -1, -1)
            x = torch.cat((cls_token, x), dim=1)

        if self.positional_emb is not None:
            x += self.positional_emb

        x = self.dropout(x)
        # transformers 堆叠
        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)

        if self.seq_pool:
            x = torch.matmul(F.softmax(self.attention_pool(x), dim=1).transpose(-1, -2), x).squeeze(-2)
        else:
            x = x[:, 0]

        x = self.fc(x)
        return x

    @staticmethod
    def init_weight(m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @staticmethod
    def sinusoidal_embedding(n_channels, dim):
        pe = torch.FloatTensor([[p / (10000 ** (2 * (i // 2) / dim)) for i in range(dim)]
                                for p in range(n_channels)])
        pe[:, 0::2] = torch.sin(pe[:, 0::2])
        pe[:, 1::2] = torch.cos(pe[:, 1::2])
        return pe.unsqueeze(0)


class CCT(nn.Module):
    def __init__(self,
                 img_size=32,
                 embedding_dim=768,
                 n_input_channels=3,
                 n_conv_layers=1,
                 kernel_size=3,
                 stride=2,
                 padding=3,
                 pooling_kernel_size=3,
                 pooling_stride=2,
                 pooling_padding=1,
                 *args, **kwargs):
        super(CCT, self).__init__()

        self.tokenizer = Tokenizer(n_input_channels=n_input_channels,
                                   n_output_channels=embedding_dim,
                                   kernel_size=kernel_size,
                                   stride=stride,
                                   padding=padding,
                                   pooling_kernel_size=pooling_kernel_size,
                                   pooling_stride=pooling_stride,
                                   pooling_padding=pooling_padding,
                                   max_pool=True,
                                   activation=nn.ReLU,  # 起码三件套得整一手吧
                                   n_conv_layers=n_conv_layers)

        self.classifier = TransformerClassifier(
            sequence_length=self.tokenizer.sequence_length(n_channels=n_input_channels,
                                                           height=img_size,
                                                           width=img_size),
            embedding_dim=embedding_dim,
            seq_pool=True,
            dropout_rate=0.,
            attention_dropout=0.1,
            stochastic_depth=0.1,
            *args, **kwargs)

    def forward(self, x):
        x = self.tokenizer(x)
        return self.classifier(x)
print('okk')

okk


In [25]:
torch.cuda.is_available() 
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
#model = ResNet(BasicBlock, [2,2,2,2], 10)
#model.to(device)

In [26]:
model = CCT(num_layers= 7,
        num_heads= 4,
        mlp_ratio= 2,
        embedding_dim= 256,
        kernel_size=3)
model.to(device)

CCT(
  (tokenizer): Tokenizer(
    (conv_layers): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 256, kernel_size=(3, 3), stride=(2, 2), padding=(3, 3), bias=False)
        (1): ReLU()
        (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      )
    )
    (flattener): Flatten(start_dim=2, end_dim=3)
  )
  (classifier): TransformerClassifier(
    (attention_pool): Linear(in_features=256, out_features=1, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0): TransformerEncoderLayer(
        (pre_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_attn): Attention(
          (qkv): Linear(in_features=256, out_features=768, bias=False)
          (attn_drop): Dropout(p=0.1, inplace=False)
          (proj): Linear(in_features=256, out_features=256, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (linear1): Linear(in_features=256, out_features=512, bias=

In [27]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-3,weight_decay=3e-2)
# scheduler = CosineAnnealingLR(optimizer, 200, 0)
criterion1 = CutMixCriterion(reduction='mean')
criterion2 = nn.CrossEntropyLoss(size_average=True)

/usr/local/miniconda3/envs/python_gpu/lib/python3.6/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [28]:
trainingloss=[]
trainingacc=[]
valloss=[]
valacc=[]
valacc5=[]

In [29]:

def get_logger(filename, verbosity=1, name=None):
    level_dict = {0: logging.DEBUG, 1: logging.INFO, 2: logging.WARNING}
    formatter = logging.Formatter(
        "[%(asctime)s][%(filename)s][line:%(lineno)d][%(levelname)s] %(message)s"
    )
    logger = logging.getLogger(name)
    logger.setLevel(level_dict[verbosity])

    fh = logging.FileHandler(filename, "w")
    fh.setFormatter(formatter)
    logger.addHandler(fh)

    sh = logging.StreamHandler()
    sh.setFormatter(formatter)
    logger.addHandler(sh)

    return logger

In [30]:
logger = get_logger('transformer终稿3/exp.log')
logger.info('start training!')

[2021-06-19 00:55:11,277][<ipython-input-30-b5afce94ecf9>][line:2][INFO] start training!
[2021-06-19 00:55:11,277][<ipython-input-30-b5afce94ecf9>][line:2][INFO] start training!


In [ ]:

epoch=300
best_accuracy=0.0
es=0
start_time=time.time()
for i in range(epoch):
    adjust_learning_rate(optimizer , i)
    model.train()
    total_loss=0.0
    accuracy=0.0
    total_correct=0.0
    total_data_num = len(train_dataloader.dataset)
    steps = 0.0
    #训练
    for batch in train_dataloader:
        steps+=1
        optimizer.zero_grad() 
        # 取数据
        inputs, labels = batch
        inputs = inputs.to(device)
        labels, shuffled_labels, lamb=labels
        labels=(labels.to(device), shuffled_labels.to(device), lamb)
        #targets1, targets2, lam = targets
        #, labels.to(device)  # 将输入和目标在每一步都送入GPU
        outputs = model(inputs)
        #_, outputs = torch.max(outputs.data, 1)
        optimizer.zero_grad()
        loss = criterion1(outputs, labels).to(device)

        loss.backward()
        optimizer.step()  

        total_loss = total_loss + loss.item() 

        #计算正确率
        labels, shuffled_labels, lamb=labels
        correct1 = (torch.max(outputs, dim=1)[1]  #get the indices
                   .view(labels.size()) == labels).sum()
        correct2 = (torch.max(outputs, dim=1)[1]  #get the indices
                   .view(shuffled_labels.size()) == shuffled_labels).sum()

        correct = (lamb * correct1.item() + (1 - lamb) * correct2.item()) 

        total_correct = total_correct + correct

 
        if steps%100==0:
            print("Epoch %d_%.3f%%:  Training average Loss: %f"
                      %(i, steps * train_dataloader.batch_size*100/len(train_dataloader.dataset),total_loss/steps))
    logger.info("Epoch %d :  Training average Loss: %f, Training accuracy: %f%%,Total Time:%f"
      %(i, total_loss/steps, total_correct*100/total_data_num,time.time()-start_time))   
    trainingloss.append(total_loss/steps)
    trainingacc.append(total_correct/total_data_num) 
    
    
    
    
    
    
    
    
    #验证
    model.eval()
    total_loss=0.0
    accuracy=0.0
    total_correct=0.0
    total_correctk=0.0
    total_data_num = len(dev_dataloader.dataset)
    steps = 0.0    
    for batch in dev_dataloader:
        steps+=1
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)  # 将输入和目标在每一步都送入GPU
        outputs = model(inputs)
        loss = criterion2(outputs, labels)  
        total_loss = total_loss + loss.item() 
        correct = (torch.max(outputs, dim=1)[1]  #get the indices
                   .view(labels.size()) == labels).sum()
        total_correct = total_correct + correct.item()

        maxk = max((1,5))
        yresize = labels.view(-1,1)
        _, pred = outputs.topk(maxk, 1, True, True)

        correctk = torch.eq(pred, yresize).sum()

        #correct = (torch.max(outputs, dim=1)[1]  #get the indices
                   #.view(labels.size()) == labels).sum()
        total_correctk = total_correctk + correctk.item()
        
    logger.info("Epoch %d :  Verification average Loss: %f, Verification accuracy: %f%%,Verification 5 accuracy: %f%%,Total Time:%f"
      %(i, total_loss/steps, total_correct*100/total_data_num,total_correctk*100/total_data_num,time.time()-start_time))  
    #print("Epoch %d :  Verification 5 accuracy: %f%%,Total Time:%f"
      #%(i,  total_correctk*100/total_data_num,time.time()-start_time))  
    valloss.append(total_loss/steps)
    valacc.append(total_correct/total_data_num) 
    valacc5.append(total_correctk/total_data_num) 
    if best_accuracy < total_correct/total_data_num :
        es = 0
        best_accuracy =total_correct/total_data_num 
        torch.save(model,'transformer终稿3/epoch_%d_accuracy_%f'%(i,total_correct/total_data_num))
        logger.info('Model is saved in transformer终稿3/epoch_%d_accuracy_%f'%(i,total_correct/total_data_num))
        #torch.cuda.empty_cache()
    #torch.save(model.state_dict(), 'cutmix/epoch_%d_accuracy_%f.pkl'%(i,total_correct/total_data_num))

Epoch 0_25.600%:  Training average Loss: 5.593862
Epoch 0_51.200%:  Training average Loss: 4.896263
Epoch 0_76.800%:  Training average Loss: 4.410850


[2021-06-19 00:55:44,389][<ipython-input-31-48b2ae7a8075>][line:51][INFO] Epoch 0 :  Training average Loss: 4.078948, Training accuracy: 10.344542%,Total Time:32.641085
[2021-06-19 00:55:44,389][<ipython-input-31-48b2ae7a8075>][line:51][INFO] Epoch 0 :  Training average Loss: 4.078948, Training accuracy: 10.344542%,Total Time:32.641085
[2021-06-19 00:55:46,580][<ipython-input-31-48b2ae7a8075>][line:92][INFO] Epoch 0 :  Verification average Loss: 2.788231, Verification accuracy: 16.080000%,Verification 5 accuracy: 62.030000%,Total Time:34.832413
[2021-06-19 00:55:46,580][<ipython-input-31-48b2ae7a8075>][line:92][INFO] Epoch 0 :  Verification average Loss: 2.788231, Verification accuracy: 16.080000%,Verification 5 accuracy: 62.030000%,Total Time:34.832413
[2021-06-19 00:55:47,985][<ipython-input-31-48b2ae7a8075>][line:102][INFO] Model is saved in transformer终稿3/epoch_0_accuracy_0.160800
[2021-06-19 00:55:47,985][<ipython-input-31-48b2ae7a8075>][line:102][INFO] Model is saved in transform

Epoch 1_25.600%:  Training average Loss: 2.671988
Epoch 1_51.200%:  Training average Loss: 2.574665
Epoch 1_76.800%:  Training average Loss: 2.507483


[2021-06-19 00:56:19,786][<ipython-input-31-48b2ae7a8075>][line:51][INFO] Epoch 1 :  Training average Loss: 2.464810, Training accuracy: 15.715503%,Total Time:68.037573
[2021-06-19 00:56:19,786][<ipython-input-31-48b2ae7a8075>][line:51][INFO] Epoch 1 :  Training average Loss: 2.464810, Training accuracy: 15.715503%,Total Time:68.037573
[2021-06-19 00:56:21,967][<ipython-input-31-48b2ae7a8075>][line:92][INFO] Epoch 1 :  Verification average Loss: 2.194917, Verification accuracy: 20.440000%,Verification 5 accuracy: 76.470000%,Total Time:70.218940
[2021-06-19 00:56:21,967][<ipython-input-31-48b2ae7a8075>][line:92][INFO] Epoch 1 :  Verification average Loss: 2.194917, Verification accuracy: 20.440000%,Verification 5 accuracy: 76.470000%,Total Time:70.218940
[2021-06-19 00:56:23,374][<ipython-input-31-48b2ae7a8075>][line:102][INFO] Model is saved in transformer终稿3/epoch_1_accuracy_0.204400
[2021-06-19 00:56:23,374][<ipython-input-31-48b2ae7a8075>][line:102][INFO] Model is saved in transform

Epoch 2_25.600%:  Training average Loss: 2.269294
Epoch 2_51.200%:  Training average Loss: 2.248110
Epoch 2_76.800%:  Training average Loss: 2.227944


[2021-06-19 00:56:55,774][<ipython-input-31-48b2ae7a8075>][line:51][INFO] Epoch 2 :  Training average Loss: 2.216511, Training accuracy: 18.734846%,Total Time:104.026144
[2021-06-19 00:56:55,774][<ipython-input-31-48b2ae7a8075>][line:51][INFO] Epoch 2 :  Training average Loss: 2.216511, Training accuracy: 18.734846%,Total Time:104.026144
[2021-06-19 00:56:57,944][<ipython-input-31-48b2ae7a8075>][line:92][INFO] Epoch 2 :  Verification average Loss: 1.962278, Verification accuracy: 27.630000%,Verification 5 accuracy: 83.900000%,Total Time:106.195740
[2021-06-19 00:56:57,944][<ipython-input-31-48b2ae7a8075>][line:92][INFO] Epoch 2 :  Verification average Loss: 1.962278, Verification accuracy: 27.630000%,Verification 5 accuracy: 83.900000%,Total Time:106.195740
[2021-06-19 00:56:59,362][<ipython-input-31-48b2ae7a8075>][line:102][INFO] Model is saved in transformer终稿3/epoch_2_accuracy_0.276300
[2021-06-19 00:56:59,362][<ipython-input-31-48b2ae7a8075>][line:102][INFO] Model is saved in trans

Epoch 3_25.600%:  Training average Loss: 2.127795
Epoch 3_51.200%:  Training average Loss: 2.113960
Epoch 3_76.800%:  Training average Loss: 2.107398


[2021-06-19 00:57:31,378][<ipython-input-31-48b2ae7a8075>][line:51][INFO] Epoch 3 :  Training average Loss: 2.100089, Training accuracy: 22.581344%,Total Time:139.630421
[2021-06-19 00:57:31,378][<ipython-input-31-48b2ae7a8075>][line:51][INFO] Epoch 3 :  Training average Loss: 2.100089, Training accuracy: 22.581344%,Total Time:139.630421
[2021-06-19 00:57:33,628][<ipython-input-31-48b2ae7a8075>][line:92][INFO] Epoch 3 :  Verification average Loss: 1.762428, Verification accuracy: 36.130000%,Verification 5 accuracy: 88.910000%,Total Time:141.880022
[2021-06-19 00:57:33,628][<ipython-input-31-48b2ae7a8075>][line:92][INFO] Epoch 3 :  Verification average Loss: 1.762428, Verification accuracy: 36.130000%,Verification 5 accuracy: 88.910000%,Total Time:141.880022
[2021-06-19 00:57:35,041][<ipython-input-31-48b2ae7a8075>][line:102][INFO] Model is saved in transformer终稿3/epoch_3_accuracy_0.361300
[2021-06-19 00:57:35,041][<ipython-input-31-48b2ae7a8075>][line:102][INFO] Model is saved in trans

Epoch 4_25.600%:  Training average Loss: 2.029927
Epoch 4_51.200%:  Training average Loss: 2.020802
Epoch 4_76.800%:  Training average Loss: 2.018031


[2021-06-19 00:58:06,884][<ipython-input-31-48b2ae7a8075>][line:51][INFO] Epoch 4 :  Training average Loss: 2.013888, Training accuracy: 26.813588%,Total Time:175.136029
[2021-06-19 00:58:06,884][<ipython-input-31-48b2ae7a8075>][line:51][INFO] Epoch 4 :  Training average Loss: 2.013888, Training accuracy: 26.813588%,Total Time:175.136029
[2021-06-19 00:58:09,912][<ipython-input-31-48b2ae7a8075>][line:92][INFO] Epoch 4 :  Verification average Loss: 1.520676, Verification accuracy: 46.440000%,Verification 5 accuracy: 92.690000%,Total Time:178.163906
[2021-06-19 00:58:09,912][<ipython-input-31-48b2ae7a8075>][line:92][INFO] Epoch 4 :  Verification average Loss: 1.520676, Verification accuracy: 46.440000%,Verification 5 accuracy: 92.690000%,Total Time:178.163906
[2021-06-19 00:58:13,572][<ipython-input-31-48b2ae7a8075>][line:102][INFO] Model is saved in transformer终稿3/epoch_4_accuracy_0.464400
[2021-06-19 00:58:13,572][<ipython-input-31-48b2ae7a8075>][line:102][INFO] Model is saved in trans

In [ ]:
trainingloss=pd.DataFrame(data=trainingloss)
trainingloss.to_csv('transformer终稿3/trainingloss.csv',encoding='utf-8')

trainingacc=pd.DataFrame(data=trainingacc)
trainingacc.to_csv('transformer终稿3/trainingacc.csv',encoding='utf-8')

valloss=pd.DataFrame(data=valloss)
valloss.to_csv('transformer终稿3/valloss.csv',encoding='utf-8')

valacc=pd.DataFrame(data=valacc)
valacc.to_csv('transformer终稿3/valacc.csv',encoding='utf-8')

valacc5=pd.DataFrame(data=valacc5)
valacc5.to_csv('transformer终稿3/valacc5.csv',encoding='utf-8')

In [18]:
import os
os.mkdir('transformer终稿3')